## In this notebook we implement some simple baselines

Including ...

In [ ]:
# Standard imports

%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import collections
import os
from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import sklearn

### Load in data 

First we load the data from NDSI, NDVI and DGA and only consider years after 1965, as there is insufficient data before 1965.

In [ ]:
processed_folder_path = os.path.join("..", "data", "processed")

In [ ]:
df_NDSI = pd.read_csv(os.path.join(processed_folder_path, "NDSI.csv"), index_col=0, parse_dates=["date"])
df_DGA = pd.read_csv(os.path.join(processed_folder_path, "DGA.csv"), index_col=0, parse_dates=["date"])
df_NDVI = pd.read_csv(os.path.join(processed_folder_path, "NDSI.csv"), index_col=0, parse_dates=["date"])

df_NDSI = df_NDSI.loc[df_NDSI["date"].dt.year >= 1965]
df_NDVI = df_NDVI.loc[df_NDVI["date"].dt.year >= 1965]
df_DGA = df_DGA.loc[df_DGA["date"].dt.year >= 1965]

In [ ]:
monthly_flow_data_mean = df_DGA.groupby(pd.PeriodIndex(df_DGA['date'], freq="M"))['river_flow'].mean()
monthly_flow_data_median = df_DGA.groupby(pd.PeriodIndex(df_DGA['date'], freq="M"))['river_flow'].median()

flow_mean_df = monthly_flow_data_mean.reset_index()
flow_mean_df.date = pd.to_datetime(flow_mean_df.date.astype("str"))

# Offset date by 3 monvary widely by jurisdiction. In many countries such as the United Kingdom, the word is not generally used and, with the exception of certain high-speed roads, there are no laws lths, so 1 april aligns with the first day of a given water year
flow_mean_df.date = flow_mean_df.date + pd.tseries.offsets.DateOffset(months=-3)

### Simple baseline using average

In [ ]:
flow_mean_df.head(5)

summer_avg = flow_mean_df.loc[flow_mean_df.date.dt.month > 6]["river_flow"].mean()
print(f"Average of all summer months: {summer_avg}")

In [ ]:
y = []

for year in flow_mean_df.date.dt.year.unique():
    year_rows = flow_mean_df.loc[flow_mean_df.date.dt.year == year]["river_flow"]
    
    if len(year_rows) == 12:
        y.append(year_rows[6:].mean())

preds = [summer_avg for _ in y]

preds = torch.Tensor(preds)
y = torch.Tensor(y)

mse_loss = nn.functional.mse_loss(preds, y).item()

print(f"If we use the average of the summer months as a predictive model, we get an MSE loss of: \n{mse_loss:.3f}")

### MLP
#### We will use Pytorch to implement some simple neural networks

https://pytorch.org/

First we refine the data and put it into a DataLoader object.

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# Process data into usable format for Pytorch
X = []
y = []

# 1987 seems to have insufficient data, so ignore years that do not contain 12 values
for year in flow_mean_df.date.dt.year.unique():
    year_rows = flow_mean_df.loc[flow_mean_df.date.dt.year == year]["river_flow"]
    
    if len(year_rows) == 12:
        X.append(year_rows[:6])
        y.append(year_rows[6:].mean())
    
X = np.array(X)
y = np.array(y)

In [ ]:
class RiverFlowDataset(Dataset):
    def __init__(self, X, y):
        self.X = np.float32(X)
        self.y = np.float32(y)
    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
class MLP(nn.Module):
    def __init__(self, inputs, outputs):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(inputs, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, outputs)
        )
        
    def forward(self, x):
        return self.layers(x)

In [ ]:
dataset = RiverFlowDataset(X, y)
train_set, val_set = torch.utils.data.random_split(dataset, [round(len(dataset) * 0.8), round(len(dataset) * 0.2)])                                              

dataloader = DataLoader(train_set, batch_size=2, shuffle=True, num_workers=2)

torch.manual_seed(42)

mlp_model = MLP(6, 1).to(device)

loss_fn = nn.MSELoss()
optim = torch.optim.Adam(mlp_model.parameters(), lr=1e-4)

for epoch in tqdm(range(0, 200), desc="Epoch"):
    
    for i, data in enumerate(dataloader):
        mlp_model.train()
        inputs, targets = data
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optim.zero_grad()
        outputs = mlp_model(inputs)
        
        loss = loss_fn(outputs, targets.unsqueeze(1))
        
        loss.backward()
        
        optim.step()

In [ ]:
def test(model, validation_set):
    testloader = DataLoader(validation_set, batch_size=2, num_workers=2)
    metric = nn.MSELoss()
    
    model.eval()
    
    total_loss = 0.0
    
    for data in tqdm(testloader):
        inputs, targets = data
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        outputs = model(inputs)
    
        total_loss += metric(outputs, targets.unsqueeze(1))
        
    test_loss = total_loss.item() / len(validation_set)
    print(f"Test Loss: {test_loss:.3f}")
    
    
# print(len(val_set))
test(mlp_model, val_set)

### RNN

To properly use an RNN we will need to do some feature engineering. Using for example the datetime as features. Furthermore we will add NDSI NDVI at some point.

First we generate out dataset.

In [ ]:
def generate_lags(df, value, n_lags):
    """
    generate_lags
    Generates a dataframe with columns denoting lagged value up to n_lags
    Args:
        df: dataframe to lag
        value: value to lag
        n_lags: amount of rows to lag
    """
    df_n = df.copy()
    
    for n in range(1, n_lags + 1):
        df_n[f"lag_{n}"] = df_n[f"{value}"].shift(n)
    
    df_n = df_n.iloc[n_lags:]
    
    return df_n

df_generated = generate_lags(flow_mean_df, "river_flow", 12)

In [ ]:
df_features = (
    df_generated
    .assign(month = df_generated.date.dt.month)
)

Given that the months in a year are cyclical we can represent the months using a sinus and cosine wave, this improves learning.

In [ ]:
def generate_cyclical_features(df, col_name, period, start_num=0):
    kwargs = {
        f"sin_{col_name}" : lambda x: np.sin(2 * np.pi * (df[col_name] - start_num) / period),
        f"cos_{col_name}" : lambda x: np.cos(2 * np.pi * (df[col_name] - start_num) / period)
    }
    
    return df.assign(**kwargs).drop(columns=[col_name])

df_features = generate_cyclical_features(df_features, "month", 12, 1)

Scaling the input data using various standard scalers improves learning

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler

def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler
    }
    
    return scalers[scaler.lower()]()

We create our own dataset which takes the features dataset and a scaler and is able to retrieve scaled datapoints.

In [ ]:
class RNN_dataset(Dataset):
    def __init__(self, df, scaler=None):
        x = df.iloc[:, 2:].values
        y = df.iloc[:, 1].values[..., np.newaxis]
        
        if scaler:
            x = scaler.fit_transform(x)
            y = scaler.fit_transform(y)
        
            self.scaler = scaler
        
        self.x_train = torch.tensor(x, dtype=torch.float32)
        self.y_train = torch.tensor(y, dtype=torch.float32)
        
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]

scaler = get_scaler("minmax")
dataset = RNN_dataset(df_features)

x, y = dataset[0]

Below we define our GRU, which was graciously taken from the blogpost: https://towardsdatascience.com/building-rnn-lstm-and-gru-for-time-series-using-pytorch-a46e5b094e7b

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [ ]:
train_set, val_set = torch.utils.data.random_split(dataset, [round(len(dataset) * 0.8), round(len(dataset) * 0.2)])                                              

input_dim = len(train_set[0][0])
hidden_dim = 64
output_dim = 1
layer_dim = 3
batch_size = 1
dropout_prob = 0.2
n_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-6

dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=2)
gru_model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim, dropout_prob)
loss_fn = nn.MSELoss(reduction="mean")
optim = torch.optim.Adam(gru_model.parameters(), lr=learning_rate, weight_decay=weight_decay)

for epoch in tqdm(range(1, n_epochs + 1), desc="Epoch"):
    for i, data in enumerate(dataloader):
        gru_model.train()
        inputs, targets = data
        inputs = inputs.view([batch_size, -1, input_dim])
        
        inputs = inputs.to(device)
        targets = targets.to(device)

        optim.zero_grad()
        outputs = gru_model(inputs)

        loss = loss_fn(outputs, targets)

        loss.backward()

        optim.step()

In [ ]:
def test_gru(model, validation_set):
    model.eval()
    total_loss = 0.0

    testloader = DataLoader(validation_set, batch_size=batch_size, num_workers=2)
    
    for i, data in enumerate(tqdm(testloader)):
        inputs, targets = data
        inputs = inputs.view([batch_size, -1, input_dim])
        
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        loss = loss_fn(model(inputs), targets)
        
        total_loss += loss.item()
                    
    print(f"Test Loss: {total_loss / len(validation_set):.3f}")

In [ ]:
test_gru(gru_model, val_set)